In [9]:
import pandas as pd

def preprocess_and_filter(df):

    df_str = df.astype(str).apply(lambda x: x.str.lower())
    
    mask_mutant = df_str.apply(lambda x: x.str.contains('mutant|mutation')).any(axis=1)
    df = df[~mask_mutant]
    
    mask_no_name = df['Enzyme Name'].isna() | df['Enzyme Name'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()
    df = df[~mask_no_name]
    
    return df

file_paths = [
    "/Volumes/dax-hd/project-data/brenda/ic50_brenda_results.csv",
    "/Volumes/dax-hd/project-data/brenda/kcat_km_brenda_results.csv",
    "/Volumes/dax-hd/project-data/brenda/ki_brenda_results.csv",
    "/Volumes/dax-hd/project-data/brenda/ph_optimum_brenda_results.csv",
    "/Volumes/dax-hd/project-data/brenda/temp_optimum_brenda_results.csv",
    "/Volumes/dax-hd/project-data/brenda/turnover_brenda_results.csv"
]

key_columns = ['EC ID', 'Enzyme Name', 'Reference']

combined_df = pd.read_csv(file_paths[0])

numerical_cols = combined_df.select_dtypes(include=['float64', 'int64']).columns

for key in key_columns:
    combined_df[key] = combined_df[key].astype(str)

for i, file in enumerate(file_paths[1:], start=1):
    df = pd.read_csv(file)
    
    for key in key_columns:
        df[key] = df[key].astype(str)
    
    df_filtered = preprocess_and_filter(df)
    
    if combined_df.empty:
        combined_df = df_filtered
    else:
        non_key_columns = df_filtered.columns.difference(key_columns)
        df_filtered.rename(columns={col: f"{col}_file{i}" for col in non_key_columns}, inplace=True)
        combined_df = pd.merge(combined_df, df_filtered, on=key_columns, how='outer')

combined_df.to_csv('/Volumes/dax-hd/project-data/brenda/brenda_combined_preprocessed.csv', index=False)

In [5]:
combined_df = pd.read_csv('/Volumes/dax-hd/project-data/brenda/brenda_combined.csv')

numerical_cols = combined_df.select_dtypes(include=['float64', 'int64']).columns
combined_df = combined_df[(combined_df[numerical_cols] >= 1).all(axis=1)]

combined_df.to_csv('/Volumes/dax-hd/project-data/brenda/brenda_preprocessed.csv', index=False)

/var/folders/fg/0p2hf3h96gz7n16ykmw22n9w0000gn/T/ipykernel_31073/198356916.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('/Volumes/dax-hd/project-data/brenda/brenda_combined.csv')


In [ ]:
from Bio.PDB import PDBList, PDBParser
import nglview as nv

# Specify the PDB ID
pdb_id = '1a0a'

# Fetch the PDB file from the PDB server
pdbl = PDBList()
filename = pdbl.retrieve_pdb_file(pdb_id, pdir='/Volumes/dax-hd/project-data/02_pdb_cleanup/1a0a.pdb', file_format='pdb')

# Parse the PDB file
parser = PDBParser()
structure = parser.get_structure(pdb_id, filename)

# Visualize the structure with NGLView
view = nv.show_biopython(structure)
view

/opt/anaconda3/envs/myenv/lib/python3.11/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 2057.
  warnings.warn(
/opt/anaconda3/envs/myenv/lib/python3.11/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 2075.
  warnings.warn(
/opt/anaconda3/envs/myenv/lib/python3.11/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2098.
  warnings.warn(
/opt/anaconda3/envs/myenv/lib/python3.11/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2121.
  warnings.warn(


NGLWidget()